In [3]:
import random
import json
import pickle
import numpy as np
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer=WordNetLemmatizer()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
with open('intents.json') as json_file:
    intents = json.load(json_file)

print(intents)


{'intents': [{'tag': 'greetings', 'patterns': ['hello', 'hey', 'hi', 'good day', 'greetings', "what's up?", 'how is it going'], 'responses': ['hello', 'hey!', 'what can i do for you?']}, {'tag': 'goodbye', 'patterns': ['cya', 'see you later', 'goodbye', 'have a good day', 'bye', 'cao', 'see ya'], 'responses': ['have a nice day', 'goodbye']}, {'tag': 'age', 'patterns': ['how old', 'how old are you?', 'what is your age', 'how old are you', 'age?'], 'responses': ['I get reborn after every compilation', 'hey!', 'my owners are averagely 20 years!']}, {'tag': 'name', 'patterns': ['what is your name', 'what should i call you', "what's your name?", 'who are you?', 'can you tell me your name'], 'responses': ['you can call me Medbot!', 'i am Medbot!', 'i am Medbot your medical assistant']}, {'tag': 'common cold symptoms', 'patterns': ['Runny or stuffy nose', 'Sore throat', 'Cough', 'Congestion', 'Slight body aches or a mild headache', 'Sneezing', 'Low-grade fever', 'Generally feeling unwell (mal

In [5]:
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']


for intent in intents['intents']:
  for pattern in intent['patterns']:
    # Convert pattern to lowercase
    pattern = pattern.lower()
    word_list=nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])



for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list=nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


words =[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes=sorted(set(classes))
pickle.dump(words,open('words2.pkl','wb'))
pickle.dump(classes,open('classes2.pkl','wb'))


In [6]:
training=[]
output_empty=[0]*len(classes)

for document in documents:
  bag=[]
  word_patterns=document[0]
  words = [lemmatizer.lemmatize(word) for word in words if word and word not in ignore_letters]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row=list(output_empty)
  output_row[classes.index(document[1])]=1
  training.append([bag,output_row])




In [7]:
random.shuffle(training)

train_x = np.array([x[0] for x in training])
train_y = np.array([x[1] for x in training])

In [8]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))


c:\Users\aksha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
 #sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
hist = model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('chatbotmodel2.h5', hist)
print('Training Done')

Epoch 1/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9921 - loss: 0.0462       
Epoch 2/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0175 
Epoch 3/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9868 - loss: 0.0718 
Epoch 4/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9962 - loss: 0.0269
Epoch 5/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0117
Epoch 6/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0185
Epoch 7/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9981 - loss: 0.0084     
Epoch 8/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9921 - loss: 0.0138     
Epoch 9/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0104     
Epoch 10/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9936 - loss: 0.0160     
Epoch 11/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9938 - loss: 0.0248 
Epoch 12/200
34/34 ━━━━━━━━━━━━━━

Training Done
